In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import json
import numpy as np
import pandas as pd
from scipy.constants import golden
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
%matplotlib inline

In [3]:
import altair as alt
from vega_datasets import data
import json

import locale
locale.setlocale(locale.LC_TIME, "it_IT")

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%config Completer. use_jedi = False

In [4]:
BASE_PATH = '../../covid19-opendata-vaccini/dati'

START_DATE = '2021-01-01'
END_DATE = datetime.today().strftime('%Y-%m-%d')

AGE_ORDER = ['12-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
AGE_ORDER = ['12-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']

FOOTNOTE = 'Grafico @RaffoRaffo85 | Dati aggiornati {}'.format(END_DATE)

In [5]:
def get_dt_given_delta(dt, delta):
    return (
        pd.to_datetime(dt) - pd.Timedelta(days=delta)
    ).strftime('%Y-%m-%d')
def get_week_range(w):
    return ' - '.join(pd.to_datetime(w).strftime('%b-%d'))

LAST_1W = get_dt_given_delta(END_DATE, 7), get_dt_given_delta(END_DATE, 1)
LAST_2W = get_dt_given_delta(END_DATE, 14), get_dt_given_delta(END_DATE, 8)

LAST_1W_STR = get_week_range(LAST_1W)
LAST_2W_STR = get_week_range(LAST_2W)

In [6]:
def get_week_cohort(dt):
    if LAST_1W[0] <= dt <= LAST_1W[1]:
        return LAST_1W_STR
    if LAST_2W[0] <= dt <= LAST_2W[1]:
        return LAST_2W_STR
    return 'rm'

In [7]:
regions = alt.topo_feature(
    'https://raw.githubusercontent.com/openpolis/geojson-italy/master/topojson/limits_IT_regions.topo.json',
    'regions'
)

# Utils

In [8]:
def extend_class(cls):
    """Source: https://gist.github.com/victorlei/5968685"""
    return lambda f: (setattr(cls,f.__name__,f) or f)

@extend_class(sns.FacetGrid)
def set_xdates(self, start, end, fmt='%B-%Y', freq='MS'):
    dt = pd.date_range(start, end, freq=freq)
    self.set(xticks=dt, xticklabels=dt.map(lambda x: x.strftime(fmt)))
    return self

@extend_class(sns.FacetGrid)
def set_suptitle(self, s, x=0, y=1.1, fontsize=18, ha='left'):
    self.fig.suptitle(s, x=x, y=y, ha=ha, fontweight='bold', fontsize=fontsize)
    return self

@extend_class(sns.FacetGrid)
def set_subtitle(self, s, x=0, y=1, c='darkgrey', fs=18, fw='bold', ha='left'):
    if isinstance(c, int):
        c = sns.color_palette('colorblind')[c]
    g.fig.text(x, y, s, color=c, fontsize=fs, fontweight=fw, ha=ha)

@extend_class(sns.FacetGrid)
def set_formatter(self, axis='xaxis', denom=1e6, fmt='{}'):
    if denom > 1:
        f = lambda x, pos: fmt.format(int(x/denom))
    else:
        f = lambda x, pos: fmt.format(x)
    for ax in self.axes.flat:
        getattr(ax, axis).set_major_formatter(FuncFormatter(f))
    return self

@extend_class(sns.FacetGrid)
def rm_y_axis(self):
    [ax.tick_params(axis='y', which=u'both',length=0) for ax in self.axes.flat]
    self.despine(left=True)
    return self

In [9]:
def collapse_to_80_plus(x):
    if x == '90+':
        return '80+'
    if x == '80-89':
        return '80+'
    return x

def get_complete_vacc(r):
    if r.fornitore == 'Janssen':
        return r.prima_dose
    return r.seconda_dose

def get_got_first_no_jansen(r):
    if r.fornitore == 'Janssen':
        return 0
    return r.prima_dose

def get_in_attesa_seconda(r):
    if r.fornitore == 'Janssen':
        return 0
    return r.prima_dose - r.seconda_dose

# Load data

In [10]:
def get_trentino_region_nome_area(nome_area):
    if 'bolzano' in nome_area.lower() or 'trento' in nome_area.lower():
        return 'Trentino-Alto Adige/Südtirol'
    if 'valle' in nome_area.lower():
        return "Valle d'Aosta"
    return nome_area
def get_trentino_region_area(area):
    if area in ['PAB', 'PAT']:
        return 'TAA'
    return area

In [11]:
# Needed to get Lat/Long to try annotate text
# def clean_friuli(x):
#     if 'friuli' in x.lower():
#         return 'Friuli-Venezia Giulia'
#     return x
# lat_long_df = (
#     pd.read_csv('../../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv')
#     [['denominazione_regione', 'lat', 'long']]
#     .drop_duplicates()
#     .assign(nome_area=lambda x: x.denominazione_regione.apply(get_trentino_region_nome_area))
#     [['nome_area', 'lat', 'long']]
#     .assign(nome_area=lambda x: x.nome_area.apply(clean_friuli))
# )

In [12]:
def clean_friuli(x):
    if x.lower().startswith('friuli'):
        return 'Friuli-Venezia Giulia'
    return x

In [13]:
lat_long_df = (
    pd.read_csv('../../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv')
    [['denominazione_regione', 'lat', 'long']]
    .drop_duplicates()
    .assign(nome_area=lambda x: x.denominazione_regione.apply(get_trentino_region_nome_area))
    [['nome_area', 'lat', 'long']]
    .drop_duplicates('nome_area')
    .assign(nome_area=lambda x: x.nome_area.apply(clean_friuli))
)
lat_long_df.head(1)

,nome_area,lat,long
0,Abruzzo,42.351222,13.398438


In [14]:
regional_wow_df = (
    pd.read_csv(os.path.join(BASE_PATH, 'somministrazioni-vaccini-latest.csv'))
    .assign(nome_area=lambda x: x.nome_area.apply(get_trentino_region_nome_area))
    .assign(area=lambda x: x.area.apply(get_trentino_region_area))
    .assign(w_cohort=lambda x: x.data_somministrazione.apply(get_week_cohort))
    .query('w_cohort!="rm"')
    .groupby(['codice_regione_ISTAT', 'nome_area', 'area', 'w_cohort'])
    [['prima_dose', 'seconda_dose']]
    .sum()
    .reset_index()
    .melt(['codice_regione_ISTAT', 'nome_area', 'area', 'w_cohort'])
    .groupby(['codice_regione_ISTAT', 'nome_area', 'area', 'w_cohort'])
    .sum().reset_index().rename(columns={'value': 'n_vaccini'})
    .merge(
        pd.read_csv(os.path.join(BASE_PATH, 'platea.csv'))
        .assign(area=lambda x: x.area.apply(get_trentino_region_area))
        .groupby('area')
        .totale_popolazione.sum().reset_index(),
        on=['area'], how='outer'
    )
    .assign(share_pop_getting_shot=lambda x: x.n_vaccini / x.totale_popolazione)
    .rename(columns={'codice_regione_ISTAT': 'reg_istat_code_num'})
    .drop(['n_vaccini', 'totale_popolazione'], axis=1)
    .merge(lat_long_df, on='nome_area', how='left')
)
assert regional_wow_df.isnull().sum().sum() == 0

In [15]:
regional_age_df = (
    pd.read_csv(os.path.join(BASE_PATH, 'somministrazioni-vaccini-latest.csv'))
    .assign(nome_area=lambda x: x.nome_area.apply(get_trentino_region_nome_area))
    .assign(area=lambda x: x.area.apply(get_trentino_region_area))
    .groupby(['fascia_anagrafica', 'codice_regione_ISTAT', 'nome_area', 'area', 'fornitore'])
    [['prima_dose', 'seconda_dose']]
    .sum()
    .reset_index()
    .assign(fascia_anagrafica=lambda x: x.fascia_anagrafica.apply(collapse_to_80_plus))
    .assign(ciclo_completo=lambda x: x.apply(get_complete_vacc, axis=1))
    .assign(solo_prima=lambda x: x.apply(get_got_first_no_jansen, axis=1))
    .assign(in_attesa_seconda=lambda x: x.apply(get_in_attesa_seconda, axis=1))
    .groupby(['fascia_anagrafica', 'area', 'codice_regione_ISTAT', 'nome_area'])
    [['ciclo_completo', 'solo_prima', 'in_attesa_seconda']]
    .sum().reset_index()
    .merge(
        pd.read_csv(os.path.join(BASE_PATH, 'platea.csv'))
        .assign(area=lambda x: x.area.apply(get_trentino_region_area))
        .groupby(['area', 'fascia_anagrafica'])
        .totale_popolazione.sum().reset_index(),
        on=['area', 'fascia_anagrafica'], how='outer'
    )
    .assign(share_ciclo_completo=lambda x: x.ciclo_completo / x.totale_popolazione)
    .assign(share_solo_prima=lambda x: x.solo_prima / x.totale_popolazione)
    .assign(share_solo_prima=lambda x: x.in_attesa_seconda / x.totale_popolazione)
    .assign(share_missing=lambda x: 1 - x.share_ciclo_completo - x.share_solo_prima)
    .rename(columns={'codice_regione_ISTAT': 'reg_istat_code_num'})
    .merge(lat_long_df, on='nome_area', how='left')
)
assert regional_age_df.isnull().sum().sum() == 0

In [16]:
REGIONS_SORTED = (
    pd.read_csv(os.path.join(BASE_PATH, 'platea.csv'))
    .assign(nome_area=lambda x: x.nome_area.apply(get_trentino_region_nome_area))
    .assign(area=lambda x: x.area.apply(get_trentino_region_area))
    .groupby(['area', 'nome_area'])
    .totale_popolazione
    .sum()
    .sort_values(ascending=False)
    .reset_index()
    [['area', 'nome_area']]
    .values
)
AREA_SORTED = [x[0] for x in REGIONS_SORTED]
REGIONS_SORTED = [x[1] for x in REGIONS_SORTED]

In [17]:
share_vax_italy = (
    regional_age_df[['ciclo_completo', 'totale_popolazione']]
    .sum()
    .to_frame()
    .T
    .assign(share_italy=lambda x: x.ciclo_completo / x.totale_popolazione)
    .share_italy.values[0]
)
share_vax_italy

0.7896774517742003

In [18]:
def concat_age_share(r):
    return r.fascia_anagrafica + ' | Media Italia: {:.0%}'.format(r.share_italy)

italy_avg_per_age_df = (
    regional_age_df
    .groupby('fascia_anagrafica', as_index=False)
    [['ciclo_completo', 'totale_popolazione']]
    .sum()
    .assign(share_italy=lambda x: x.ciclo_completo / x.totale_popolazione)
    [['fascia_anagrafica', 'share_italy']]
    .assign(cohort=lambda x: x.apply(concat_age_share, axis=1))
    [['fascia_anagrafica', 'cohort']]
)

# Share vaccinated by age overall

In [19]:
# Share vaccinated by age overall
str_rename = {'share_ciclo_completo': 'Completo', 'share_solo_prima': 'In attesa seconda', 'share_missing': 'Nessun vaccino'}.get
order_rename = {'share_ciclo_completo': 1, 'share_solo_prima': 0, 'share_missing': 2}.get
plt_share_vacc_overall_df = (
    regional_age_df
    .groupby('fascia_anagrafica')
    [['ciclo_completo', 'solo_prima', 'in_attesa_seconda', 'totale_popolazione']]
    .sum()
    .reset_index()
    .assign(share_solo_prima=lambda x: x.in_attesa_seconda/x.totale_popolazione)
    .assign(share_ciclo_completo=lambda x: x.ciclo_completo/x.totale_popolazione)
    .assign(share_missing=lambda x: 1 - x.share_solo_prima - x.share_ciclo_completo)
    .melt(
        id_vars=['fascia_anagrafica'],
        value_vars=['share_ciclo_completo', 'share_solo_prima', 'share_missing']
    )
    .assign(Vaccino=lambda x: x.variable.apply(str_rename))
    .assign(order=lambda x: x.variable.apply(order_rename))
)
plot_title = alt.TitleParams(
    "Percentuale di popolazione per ciclo vaccinale per fascia anagrafica",
    fontSize=18,
    subtitle=[FOOTNOTE],
    subtitleColor='grey', subtitleFontSize=16,
    align='left', anchor='start'
)
(
    alt.Chart(plt_share_vacc_overall_df, title=plot_title)
    .mark_bar()
    .encode(
        x=alt.X('value:Q', stack='zero', axis=alt.Axis(format='.0%'), title=''),
        y=alt.Y('fascia_anagrafica:N', title=''),
        color=alt.Color('Vaccino:N', sort=alt.EncodingSortField('order', order='ascending')),
        order='order',
        tooltip=[
            alt.Tooltip('fascia_anagrafica:N', title='Fascia anagrafica'),
            alt.Tooltip('Vaccino:N', title='Vaccino'),
            alt.Tooltip('value:Q', format='.1%', title='Popolazione vaccinata')
        ]
    )
    .properties(width=200*golden, height=200)
    .configure_view(strokeOpacity=0)
)

alt.Chart(...)

# Regional share

In [20]:
str_rename = {'share_ciclo_completo': 'Completo', 'share_solo_prima': 'In attesa seconda', 'share_missing': 'Nessun vaccino'}.get
order_rename = {'share_ciclo_completo': 1, 'share_solo_prima': 0, 'share_missing': 2}.get
plot_title = alt.TitleParams(
    "Percentuale di popolazione per ciclo vaccinale per fascia anagrafica",
    fontSize=18,
    subtitle=[FOOTNOTE],
    subtitleColor='grey', subtitleFontSize=16,
    align='left', anchor='start'
)
bar_plt_df = (
    regional_age_df
    .melt(
        id_vars=['fascia_anagrafica', 'nome_area'],
        value_vars=['share_ciclo_completo', 'share_solo_prima', 'share_missing']
    )
    .assign(Vaccino=lambda x: x.variable.apply(str_rename))
    .assign(order=lambda x: x.variable.apply(order_rename))
    #.query('nome_area=="Campania"')
)
(
    alt.Chart(bar_plt_df, title=plot_title)
    .mark_bar()
    .encode(
        x=alt.X('value:Q', stack='zero', axis=alt.Axis(format='.0%'), title=''),
        y=alt.Y('fascia_anagrafica:N', title=''),
        color=alt.Color('Vaccino:N', sort=alt.EncodingSortField('order', order='ascending')),
        order='order',
        facet=alt.Facet('nome_area:N', columns=7, title=''),
        tooltip=[
            alt.Tooltip('fascia_anagrafica:N', title='Fascia anagrafica'),
            alt.Tooltip('Vaccino:N', title='Vaccino'),
            alt.Tooltip('value:Q', format='.1%', title='Popolazione vaccinata')
        ]
    )
    .properties(width=120, height=120)
    .configure_view(strokeOpacity=0)
)

alt.Chart(...)

# Maps

In [21]:
single_region_plot_df = (
    regional_age_df
    .groupby(['area', 'nome_area', 'reg_istat_code_num'])
    [['ciclo_completo', 'totale_popolazione']]
    .sum()
    .reset_index()
    .assign(share_ciclo_completo=lambda x: x.ciclo_completo/x.totale_popolazione)
    .drop(['ciclo_completo', 'totale_popolazione'], axis=1)
)
single_region_plot_df.head(1)

,area,nome_area,reg_istat_code_num,share_ciclo_completo
0,ABR,Abruzzo,13,0.782015


## Italy overall

In [22]:
single_region_plot_df[['share_ciclo_completo']].describe().T[['min', 'max']]

,min,max
share_ciclo_completo,0.72494,0.827507


In [23]:
# The scale is fixed 6% to 8% so make sure it makes sense
assert single_region_plot_df.share_ciclo_completo.min() >= 0.7009, f'{single_region_plot_df.share_ciclo_completo.min()}'
assert single_region_plot_df.share_ciclo_completo.min() < 0.74, f'{single_region_plot_df.share_ciclo_completo.min()}'
assert single_region_plot_df.share_ciclo_completo.max() <= 0.851, f'{single_region_plot_df.share_ciclo_completo.max()}'
assert single_region_plot_df.share_ciclo_completo.max() > 0.78, f'{single_region_plot_df.share_ciclo_completo.max()}'
plot_title = alt.TitleParams(
    f"Popolazione con ciclo vaccinale completo. Media Italia: {share_vax_italy:.0%}",
    fontSize=18,
    subtitle=[FOOTNOTE],
    subtitleColor='grey', subtitleFontSize=16,
    align='left', anchor='start'
)
(
    alt.Chart(single_region_plot_df, title=plot_title)
    .mark_geoshape()
    .encode(
        shape='geo:G',
        color=alt.Color(
            field='share_ciclo_completo',
            type='quantitative',
            title='Ciclo completo',
            scale=alt.Scale(domain=[.7, .85]),
            legend=alt.Legend(format=".0%")
            # format='.0%'
        ),
        tooltip=[
            alt.Tooltip(field='nome_area', type='nominal', title='Region'),
            alt.Tooltip(field='share_ciclo_completo', type='quantitative', format='.1%', title='Ciclo completo')
        ],
    )
    .transform_lookup(
        lookup='reg_istat_code_num',
        from_=alt.LookupData(data=regions, key='properties.reg_istat_code_num'),
        as_='geo'
    )
    .properties(width=400, height=500)
    .configure_view(strokeOpacity=0)
)

alt.Chart(...)

## Week over week overall

In [24]:
# The scale is fixed 1% to 5% so make sure it makes sense
assert regional_wow_df.share_pop_getting_shot.max() <= 0.025, 'Change scale! max={:.2}'.format(regional_wow_df.share_pop_getting_shot.max())
assert regional_wow_df.share_pop_getting_shot.min() >= 0.0074, 'Change scale! min={:.2}'.format(regional_wow_df.share_pop_getting_shot.min())
assert regional_wow_df.share_pop_getting_shot.max() > 0.018, 'Change scale! max={:.2}'.format(regional_wow_df.share_pop_getting_shot.max())
assert regional_wow_df.share_pop_getting_shot.min() < 0.0087, 'Change scale! min={:.2}'.format(regional_wow_df.share_pop_getting_shot.min())


plot_title = alt.TitleParams(
    "Percentuale popolazione ricevente 1a o 2a dose nelle ultime 2 settimane",
    fontSize=18,
    subtitle=[FOOTNOTE],
    subtitleColor='grey', subtitleFontSize=16,
    align='left', anchor='start'
)
(
    alt.Chart(regional_wow_df, title=plot_title)
    .mark_geoshape()
    .encode(
        shape='geo:G',
        color=alt.Color(
            field='share_pop_getting_shot',
            type='quantitative',
            title='Vaccini, %',
            scale=alt.Scale(domain=[.005, .025]),
            legend=alt.Legend(format=".1%")
        ),
        facet=alt.Facet('w_cohort:N', header=alt.Header(labelFontSize=15, labelFontWeight='bold', title='')),
        tooltip=[
            alt.Tooltip(field='w_cohort', type='nominal', title='Settimana'),
            alt.Tooltip(field='nome_area', type='nominal', title='Region'),
            alt.Tooltip(field='share_pop_getting_shot', type='quantitative', format='.1%', title='Ciclo completo')
        ],
    )
    .transform_lookup(
        lookup='reg_istat_code_num',
        from_=alt.LookupData(data=regions, key='properties.reg_istat_code_num'),
        as_='geo'
    )
    .properties(width=200, height=300)
    .configure_view(strokeOpacity=0)
)

alt.Chart(...)

## Italy per age

In [25]:
plot_title = alt.TitleParams(
    f"Percentuale di popolazione con ciclo vaccinale completo per fascia anagrafica (media Italia: {share_vax_italy:.0%})",
    fontSize=18,
    subtitle=[FOOTNOTE],
    subtitleColor='grey', subtitleFontSize=16,
    align='left', anchor='start'
)
(
    alt.Chart(regional_age_df.merge(italy_avg_per_age_df), title=plot_title)
    .mark_geoshape()
    .encode(
        shape='geo:G',
        color=alt.Color(
            field='share_ciclo_completo',
            type='quantitative',
            title='Ciclo completo',
            legend=alt.Legend(format=".0%", clipHeight=.2)
        ),
        facet=alt.Facet(
            'cohort:N', columns=4,
            header=alt.Header(title='', labelFontSize=15, labelFontWeight='bold')
        ),
        tooltip=[
            alt.Tooltip(field='nome_area', type='nominal', title='Region'),
            alt.Tooltip(field='fascia_anagrafica', type='nominal', title='Fascia anagrafica'),
            alt.Tooltip(field='share_ciclo_completo', type='quantitative', format='.0%', title='Ciclo completo')
        ],
    )
    .transform_lookup(
        lookup='reg_istat_code_num',
        from_=alt.LookupData(data=regions, key='properties.reg_istat_code_num'),
        as_='geo'
    )
    .properties(width=130, height=200)
    .configure_view(strokeOpacity=0)
    .resolve_scale(color='independent')
    .configure_legend(
        gradientLength=150,
        gradientThickness=10
    ) 
)

alt.Chart(...)